# ChatGPT con la API: Ejemplo chatbot mínimo

## Imports y configuración de la API key

In [3]:
!pip install --upgrade openai --quiet
import os
from openai import OpenAI

# Pegar acá la API key
# Tomar de acá: https://platform.openai.com/account/api-keys
# Cuidado, no dejar pública!!
API_KEY = 'sk-...'

## Clase para Chatbot multifunción

Para los que quieran profundizar, [acá](https://platform.openai.com/docs/guides/chat/introduction) está la documentación.

In [48]:
class ChatBotGPT:
    def __init__(self, rol):
        self.rol = rol
        self.mensajes = [{"role": "system", "content": rol}]
        self.cliente = OpenAI(api_key=API_KEY)

    def enviar_mensaje(self, mensaje):
        self.mensajes.append({'role': 'user', 'content': mensaje})
        resultado = self.cliente.chat.completions.create(model="gpt-3.5-turbo", messages=self.mensajes)
        mensaje_recibido = resultado.choices[0].message
        self.mensajes.append({'role': mensaje_recibido.role, 'content': mensaje_recibido.content})
        print(f"Usuario: {mensaje}")
        print(f"ChatGPT: {mensaje_recibido.content}")

    def mostrar_interaccion_completa(self):
        for mensaje in self.mensajes:
            if mensaje['role'] == 'system':
                continue
            msj = mensaje['content']
            agente = 'Usuario' if mensaje['role'] == 'user' else 'ChatGPT'
            print(f"{agente}: {msj}")





## Ejemplo: vendedor de productos Apple

In [49]:
productos_tecnologicos = [
    {'producto': 'iPhone 6', 'lanzamiento': 2007, 'precio': 300},
    {'producto': 'iPhone 14', 'lanzamiento': 2022, 'precio': 1200},
    {'producto': 'iPhone 14 pro', 'lanzamiento': 2022, 'precio': 1300},
    {'producto': 'iPad', 'lanzamiento': 2010, 'precio': 799},
    {'producto': 'Apple Watch', 'lanzamiento': 2015, 'precio': 399},
    {'producto': 'iMac', 'lanzamiento': 1998, 'precio': 1099},
    {'producto': 'MacBook Pro', 'lanzamiento': 2006, 'precio': 1299}
]


rol_asistente = f"""Quiero que actues como un agente conversacional para mi negocio.
Vendo productos tecnológicos y este es mi catálogo: {productos_tecnologicos}"""

vendedor_apple = ChatBotGPT(rol_asistente)

In [50]:
vendedor_apple.enviar_mensaje("Hola, qué tal? Quiero comprar un producto")

Usuario: Hola, qué tal? Quiero comprar un producto
ChatGPT: ¡Hola! ¿En qué puedo ayudarte? ¿Qué producto te interesa comprar?


In [51]:
vendedor_apple.enviar_mensaje("Quiero un iPhone")

Usuario: Quiero un iPhone
ChatGPT: Perfecto, tenemos varios modelos de iPhones disponibles. ¿Estás buscando algún modelo en particular o tienes alguna preferencia de lanzamiento o precio en mente?


Acá se sale del libreto. Quizás podríamos mejorar el rol?

In [52]:
vendedor_apple.enviar_mensaje("Dispongo de 500 dólares. Qué puedo comprar?")

Usuario: Dispongo de 500 dólares. Qué puedo comprar?
ChatGPT: Con un presupuesto de 500 dólares, tienes varias opciones disponibles. Puedes adquirir un iPhone 6 por $300 o un iPhone 14 por $1200. Si estás interesado en los modelos más recientes, también puedes considerar el iPhone 14 Pro, que tiene un precio de $1300. ¿Te gustaría más información sobre alguno de estos modelos?


In [53]:
vendedor_apple.enviar_mensaje("Entiendo. Dejame pensarlo. Mientras tanto, quién ganó el mundial de futbol de 1986?")

Usuario: Entiendo. Dejame pensarlo. Mientras tanto, quién ganó el mundial de futbol de 1986?
ChatGPT: El ganador del Mundial de Fútbol de 1986 fue la selección nacional de Argentina. Fue un torneo muy destacado para Argentina, donde su capitán Diego Maradona tuvo una destacada actuación.


In [54]:
vendedor_apple.mostrar_interaccion_completa()


Usuario: Hola, qué tal? Quiero comprar un producto
ChatGPT: ¡Hola! ¿En qué puedo ayudarte? ¿Qué producto te interesa comprar?
Usuario: Quiero un iPhone
ChatGPT: Perfecto, tenemos varios modelos de iPhones disponibles. ¿Estás buscando algún modelo en particular o tienes alguna preferencia de lanzamiento o precio en mente?
Usuario: Dispongo de 500 dólares. Qué puedo comprar?
ChatGPT: Con un presupuesto de 500 dólares, tienes varias opciones disponibles. Puedes adquirir un iPhone 6 por $300 o un iPhone 14 por $1200. Si estás interesado en los modelos más recientes, también puedes considerar el iPhone 14 Pro, que tiene un precio de $1300. ¿Te gustaría más información sobre alguno de estos modelos?
Usuario: Entiendo. Dejame pensarlo. Mientras tanto, quién ganó el mundial de futbol de 1986?
ChatGPT: El ganador del Mundial de Fútbol de 1986 fue la selección nacional de Argentina. Fue un torneo muy destacado para Argentina, donde su capitán Diego Maradona tuvo una destacada actuación.


In [59]:
def get_completion_from_messages(messages):
    cliente = OpenAI(api_key=API_KEY)
    response = cliente.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('Usuario:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Asistente:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content' : rol_asistente} ]  # context va a ir acumulando los mensajes


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard